In [1]:
import gym
import random
import numpy as np
from collections import deque

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model

from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


#### Implementação original:
https://ieeexplore-ieee-org.ezp2.lib.umn.edu/stamp/stamp.jsp?tp=&arnumber=6313077

#### Baseado no jupyter-notebook: 
https://github.com/gsurma/cartpole/blob/master/cartpole.py

In [2]:

ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995


In [3]:
class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = tf.keras.Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Tomar Decisões Aleatorias em certos momentos
        if np.random.rand() < self.exploration_rate:
            print("Inovou.")
            return random.randrange(self.action_space)
        # Prever q_value dado o estado atual
        q_values = self.model.predict(state)
        # Retornar acao
        return np.argmax(q_values[0])

    def experience_replay(self):
        # Retreinar modelo com novas informacoes
        if len(self.memory) < BATCH_SIZE:
            return
        # Selecionar aleatoriamente momentos na memoria
        batch = random.sample(self.memory, BATCH_SIZE)
        # Para cada momento na memoria
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                # Caso o estado não tenha sido terminal
                # Atualiza q_values. Ver imagem: https://wikimedia.org/api/rest_v1/media/math/render/svg/678cb558a9d59c33ef4810c9618baf34a9577686
                # Novo q_value é Recompensa + Com a ação prevista para o próximo estado * Gamma
                # Se a recompensa for negativa, o q_value para esta ação no proximo estado é diminuido.
                # Se a recompensa for positiva, o q_value para esta ação no proximo estado é aumentado
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0) #Online learning - O modelo é retreinado a cada momento lembrado
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

###  Ideia da solução baseada em reforço

<img src="https://miro.medium.com/max/1400/1*mPGk9WTNNvp3i4-9JFgD3w.png" width=600 />

In [4]:
def cartpole():
    #Iniciar o ambiente de treino
    env = gym.make("CartPole-v1")
    #Salvar tamanho do espaço observado - Neste caso é 4 (Posicao do carro, Velocidade do Carro, Angulo com vertical, Velocidade do Topo)
    observation_space = env.observation_space.shape[0]
    # Salvar tamanho do espaco de acao. - Neste caso é 2 (esquerda ou direita)
    action_space = env.action_space.n
    # Inicializa nosso resolvedor
    dqn_solver = DQNSolver(observation_space, action_space)
    episode = 0
    while True:
        steps = 0
        episode += 1
        # Reinicia o ambiente a cada interacao
        state = env.reset()
        # Transforma o vetor state em matrix
        state = np.reshape(state, [1, observation_space])
        
        while True:
            # Desenha o ambiente
            env.render()
            # Preve melhor proxima acao dado estado corrente
            action = dqn_solver.act(state)
            # Realiza acao e atualiza status
            state_next, reward, terminal, info = env.step(action)
            # Se o ambiente não foi finalizado a recompensa é positiva, se não negativa.
            reward = reward if not terminal else -reward
            # Transforma o vetor state em matrix
            state_next = np.reshape(state_next, [1, observation_space])
            q_values = dqn_solver.model.predict(state)
            print("\tSTEP: {} - Valores Q: {}\n\tAcao: {} Recompensa: {} Finalizado:{}\n\tPROXIMO ESTADO- Pos: {:.2f} Car Vel:{:.2f} Angulo:{:.2f} Vel Top:{:.2f}".format(steps,q_values, action, reward, terminal, state_next[0][0],state_next[0][1],state_next[0][2],state_next[0][3]))
            # Faz com que a acao seja lembrada.
            dqn_solver.remember(state, action, reward, state_next, terminal)
            # Re-treina nosso modelo com a nova experiencia
            dqn_solver.experience_replay()
            state = state_next
            steps += 1
            if terminal:
                break
        print("Episode: {} - Passos no ultimo episodio: {}".format(episode, steps))


In [5]:
cartpole()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'